In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

load_dotenv()

client = OpenAI(api_key=os.environ["LLM_API_KEY"])
MODEL_NAME = "gpt-4o-mini"

"""
1行（1人分）の情報から、資格取得計画を提案してdictで返す
"""
def propose_cert_plan(row: pd.Series) -> dict:
    prompt_text = f"""
    あなたはIT研修のメンターです。
    以下の社員プロフィールをもとに、半年〜1年を目安に「おすすめ資格」と「学習順」を提案してください。

    # 社員プロフィール
    社員名: {row.get("社員名", "")}
    現在スキル: {row.get("現在スキル", "")}
    経験年数: {row.get("経験年数", "")}
    目標職種: {row.get("目標職種", "")}
    希望分野: {row.get("希望分野", "")}
    週あたり学習時間: {row.get("週あたり学習時間", "")}

    # 出力ルール（必ずJSONで）
    {{
    "おすすめ資格": ["資格1", "資格2", "資格3"],
    "学習順": ["ステップ1", "ステップ2", "ステップ3"],
    "理由": "短く要点だけ",
    "ざっくり学習計画": "週あたり学習時間を踏まえたざっくり計画"
    }}

    注意:
    - 資格名は正式名称っぽく
    - 3つまで
    - できるだけ現実的に
    """

    try:
        response = client.chat.completions.create(
            model=MODEL_NAME,
            messages=[{"role": "user", "content": prompt_text}],
            temperature=0.3,
            response_format={"type": "json_object"},
        )
        data = json.loads(response.choices[0].message.content)
        return data
    except Exception as e:
        print(f"API呼び出しでエラーが発生しました: {e}")
        return {
            "おすすめ資格": [],
            "学習順": [],
            "理由": "エラー",
            "ざっくり学習計画": "エラー",
        }

print("処理を開始します。")

df = pd.read_excel("社員情報.xlsx", sheet_name="資格計画")

results = df.apply(propose_cert_plan, axis=1)

# dictの列をDataFrame化して結合
df_out = pd.concat([df, results.apply(pd.Series)], axis=1)

# リスト列はExcelで見やすいように文字列化
for col in ["おすすめ資格", "学習順"]:
    df_out[col] = df_out[col].apply(lambda x: " / ".join(x) if isinstance(x, list) else str(x))

df_out.to_excel("資格提案.xlsx", index=False)

print("Excelファイルに結果を保存しました。")

処理を開始します。
Excelファイルに結果を保存しました。
